In [1]:
!git clone https://github.com/Adamouization/Content-Based-Video-Retrieval-Code.git

Cloning into 'Content-Based-Video-Retrieval-Code'...
remote: Enumerating objects: 908, done.
remote: Total 908 (delta 0), reused 0 (delta 0), pack-reused 908
Receiving objects: 100% (908/908), 418.20 MiB | 30.51 MiB/s, done.
Resolving deltas: 100% (400/400), done.
Checking out files: 100% (247/247), done.


In [2]:
%cd /content/Content-Based-Video-Retrieval-Code

/content/Content-Based-Video-Retrieval-Code


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 768kB 4.9MB/s 
     |████████████████████████████████| 184kB 10.2MB/s 
     |████████████████████████████████| 952kB 12.6MB/s 
     |████████████████████████████████| 12.9MB 329kB/s 
     |████████████████████████████████| 17.3MB 250kB/s 
     |████████████████████████████████| 31.2MB 139kB/s 
     |████████████████████████████████| 25.4MB 107kB/s 
     |████████████████████████████████| 10.1MB 40.0MB/s 
     |████████████████████████████████| 24.8MB 1.6MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
  Created wheel for backcall: filename=backcall-0.1.0-cp36-none-any.whl size=10414 sha256=80a5a8be267fcbf2688c8542453277bcc1184b13b2fed7693bbc8e1ab74d8f9e
  Stored in directory: /root/.cache/pip/wheels/98/b0/dd/29e28ff615af3dda4c67cab719dd51357597eabff926976b45
  Created wheel for imutils: filename=imutils-0.5.2-cp36-none-any.whl size=24417 sha256=58287c6b295f03b745a8249d84825bf542a0187479cf5840db27f6e36d33e4ff
  Stored in directory: /roo

In [7]:
!ls

app  footage  LICENSE  README.md  recordings  requirements.txt	results


In [13]:
import app.config as config
#from app.helpers import display_results_histogram, get_number_of_frames, get_video_filenames, get_video_fps, \
    #get_video_first_frame, print_finished_training_message, terminal_yes_no_question, show_final_match, \
    #video_file_already_stabilised
from app.helpers import display_results_histogram
from app.helpers import get_number_of_frames
from app.helpers import get_video_filenames
from app.helpers import get_video_fps
from app.helpers import get_video_first_frame
from app.helpers import  print_finished_training_message
from app.helpers import terminal_yes_no_question
from app.helpers import show_final_match
from app.helpers import video_file_already_stabilised
    #get_video_first_frame, print_finished_training_message, terminal_yes_no_question, show_final_match, \
    #video_file_already_stabilised
from app.histogram import HistogramGenerator
from app.video_operations import VideoStabiliser


In [15]:
!python /content/Content-Based-Video-Retrieval-Code/app/main.py --model all --mode test

Traceback (most recent call last):
  File "/content/Content-Based-Video-Retrieval-Code/app/main.py", line 11, in <module>
    from app.helpers import display_results_histogram
ModuleNotFoundError: No module named 'app'


In [ ]:
import argparse
from collections import Counter
import time

from pyspin.spin import make_spin, Spin2

import app.config as config
from app.helpers import display_results_histogram, get_number_of_frames, get_video_filenames, get_video_fps, \
    get_video_first_frame, print_finished_training_message, terminal_yes_no_question, show_final_match, \
    video_file_already_stabilised
from app.histogram import HistogramGenerator
from app.video_operations import VideoStabiliser


def main():
    """
    Program entry point. Parses command line input to decide which phase of the system to run.
    :return: None
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("-m", "--model",
                        help="The histogram model to use. Choose from the following options: 'rgb', 'hsv' or 'gray'. "
                             "Leave empty to train using all 3 histogram models.")
    parser.add_argument("--mode",
                        required=True,
                        help="The mode to run the code in. Choose from the following options: 'train', 'test' or "
                             "'segment'.")
    parser.add_argument("--showhists",
                        action="store_true",
                        help="Specify whether you want to display each generated histogram.")
    parser.add_argument("-d", "--debug",
                        action="store_true",
                        help="Specify whether you want to print additional logs for debugging purposes.")
    args = parser.parse_args()
    config.debug = args.debug
    config.mode = args.mode
    config.show_histograms = args.showhists
    config.model = args.model

    if config.mode == "train":
        off_line_colour_based_feature_extraction_phase()
    elif config.mode == "test":
        on_line_retrieval_phase()
    elif config.mode == "segment":
        database_preprocessing_phase()
    else:
        print("Wrong mode chosen. Choose from the following options: 'train', 'test' or 'segment'.")
        exit(0)


@make_spin(Spin2, "Generating histograms for database videos...".format(config.model))
def off_line_colour_based_feature_extraction_phase():
    """
    Generates and stores averaged greyscale, RGB and HSV histograms for all the videos in the directory-based database.
    :return: None
    """
    directory = "../footage/"
    files = get_video_filenames(directory)

    # start measuring runtime
    start_time = time.time()

    for file in files:
        if config.model == "gray":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_greyscale_histogram()
        elif config.model == "rgb":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_rgb_histogram()
        elif config.model == "hsv":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_hsv_histogram()
        else:
            histogram_generator_gray = HistogramGenerator(directory, file)
            histogram_generator_gray.generate_video_greyscale_histogram()
            histogram_generator_rgb = HistogramGenerator(directory, file)
            histogram_generator_rgb.generate_video_rgb_histogram()
            histogram_generator_hsv = HistogramGenerator(directory, file)
            histogram_generator_hsv.generate_video_hsv_histogram()
    runtime = round(time.time() - start_time, 2)
    print_finished_training_message(config.model, directory, runtime)


def on_line_retrieval_phase():
    """
    Prompts the user to stabilise and crop the query video before generating the same averaged greyscale, RGB and HSV
    histograms to compare with the database videos' previously stored histograms using distance metrics.
    :return: None
    """
    directory = "../recordings/"
    recordings = ["recording1.mp4", "recording2.mp4", "recording3.mp4", "recording4.mp4", "recording5.mp4",
                  "recording6.mp4", "recording7.mp4", "recording8.mp4"]
    mismatches_directory = "../recordings/mismatches/"
    mismatches = ["mismatch1.mp4", "mismatch2.mp4"]
    # 0: cloudy-sky, 1: seal, 2: butterfly (skewed), 3: wind-turbine, 4: ice-hockey, 5: jellyfish, 6: people-dancing,
    # 7: jellyfish (skewed)
    file = recordings[7]

    # ask user to stabilise the input query video or not
    is_stabilise_video = terminal_yes_no_question("Do you wish to stabilise the recorded query video?")
    stable_filename = "stable-" + file[:-4] + ".avi"  # the stable version of the video
    # yes: stabilise the video and use the stable .avi version
    if is_stabilise_video:
        if not video_file_already_stabilised(directory + stable_filename):
            VideoStabiliser(directory, "{}".format(file))
        print("\nStabilised version of query already found: '{}'".format(stable_filename))
        file = stable_filename
    # no: check if a version of the stabilised video doesn't already exist - use it if it does
    else:
        if video_file_already_stabilised(directory + stable_filename):
            file = stable_filename

    print("\nUsing query: '{}'".format(file))
    print("\nPlease crop the recorded query video for the signature to be generated.")

    if config.model == "gray":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_greyscale_histogram(is_query=True)
        histogram_generator.match_histograms()
    elif config.model == "rgb":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_rgb_histogram(is_query=True)
        histogram_generator.match_histograms()
    elif config.model == "hsv":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_hsv_histogram(is_query=True)
        histogram_generator.match_histograms()
    else:
        # calculate query histogram
        # greyscale
        histogram_generator_gray = HistogramGenerator(directory, file)
        histogram_generator_gray.generate_video_greyscale_histogram(is_query=True)
        cur_reference_points = histogram_generator_gray.get_current_reference_points()
        # start measuring runtime (after manual cropping)
        start_time = time.time()
        # RGB
        histogram_generator_rgb = HistogramGenerator(directory, file)
        histogram_generator_rgb.generate_video_rgb_histogram(is_query=True, cur_ref_points=cur_reference_points)
        # HSV
        histogram_generator_hsv = HistogramGenerator(directory, file)
        histogram_generator_hsv.generate_video_hsv_histogram(is_query=True, cur_ref_points=cur_reference_points)

        # calculate distances between query and DB histograms
        histogram_generator_gray.match_histograms(cur_all_model='gray')
        histogram_generator_rgb.match_histograms(cur_all_model='rgb')
        histogram_generator_hsv.match_histograms(cur_all_model='hsv')

        # Combine matches from all 3 histogram models to output one final result
        all_results = histogram_generator_hsv.get_results_array()  # array of all matches made (using weights)
        results_count = Counter(all_results)  # count the number of matches made for each video in all_results array
        # transform from count to percentage of matches made
        results_percentage = dict()
        for match in results_count:
            percentage = round((results_count[match] / len(all_results)) * 100.0, 2)
            results_percentage[match] = percentage
        display_results_histogram(results_percentage)
        print("Matches made: {}".format(results_count))
        print("% of matches made: {}".format(results_percentage))

        # find best result
        final_result_name = ""
        final_result_count = 0
        for i, r in enumerate(results_count):
            if i == 0:
                final_result_name = r
                final_result_count = results_count[r]
            else:
                if results_count[r] > final_result_count:
                    final_result_name = r
                    final_result_count = results_count[r]

        # print results
        runtime = round(time.time() - start_time, 2)
        accuracy = final_result_count / len(all_results)
        get_video_first_frame(directory + file, "../results", is_query=True)
        get_video_first_frame("../footage/{}".format(final_result_name), "../results", is_result=True)
        show_final_match(final_result_name, "../results/query.png", "../results/result.png", runtime, accuracy)
        print_finished_training_message(final_result_name, config.model, runtime, accuracy)


def database_preprocessing_phase():
    """
    Applies a shot boundary detection algorithm to a video for segmentation.
    :return: None
    """
    directory = "../recordings/"
    video = "scene-segmentation.mp4"
    # directory = "/Volumes/ADAM2/"
    # movies = ["Inception (2010).mp4"]
    # video = movies[0]

    shot_boundary_detector = HistogramGenerator(directory, video)
    video_capture = shot_boundary_detector.get_video_capture()
    frame_count = get_number_of_frames(vc=video_capture)
    fps = get_video_fps(vc=video_capture)
    print("Total Frames: {}".format(frame_count))
    print("FPS: {}\n".format(fps))

    # start measuring runtime
    start_time = time.time()

    # start processing video for shout boundary detection
    print("Starting to process video for shot boundary detection...")
    shot_boundary_detector.rgb_histogram_shot_boundary_detection(threshold=7)

    # print final results
    runtime = round(time.time() - start_time, 2)
    print("--- Number of frames in video: {} ---".format(frame_count))
    print("--- Runtime: {} seconds ---".format(runtime))


if __name__ == "__main__":
    main()


In [ ]:
import argparse
from collections import Counter
import time

from pyspin.spin import make_spin, Spin2

import config as config
from helpers import display_results_histogram, get_number_of_frames, get_video_filenames, get_video_fps, \
    get_video_first_frame, print_finished_training_message, terminal_yes_no_question, show_final_match, \
    video_file_already_stabilised
from histogram import HistogramGenerator
from video_operations import VideoStabiliser


def main():
    """
    Program entry point. Parses command line input to decide which phase of the system to run.
    :return: None
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("-m", "--model",
                        help="The histogram model to use. Choose from the following options: 'rgb', 'hsv' or 'gray'. "
                             "Leave empty to train using all 3 histogram models.")
    parser.add_argument("--mode",
                        required=True,
                        help="The mode to run the code in. Choose from the following options: 'train', 'test' or "
                             "'segment'.")
    parser.add_argument("--showhists",
                        action="store_true",
                        help="Specify whether you want to display each generated histogram.")
    parser.add_argument("-d", "--debug",
                        action="store_true",
                        help="Specify whether you want to print additional logs for debugging purposes.")
    args = parser.parse_args()
    config.debug = args.debug
    config.mode = args.mode
    config.show_histograms = args.showhists
    config.model = args.model

    if config.mode == "train":
        off_line_colour_based_feature_extraction_phase()
    elif config.mode == "test":
        on_line_retrieval_phase()
    elif config.mode == "segment":
        database_preprocessing_phase()
    else:
        print("Wrong mode chosen. Choose from the following options: 'train', 'test' or 'segment'.")
        exit(0)


@make_spin(Spin2, "Generating histograms for database videos...".format(config.model))
def off_line_colour_based_feature_extraction_phase():
    """
    Generates and stores averaged greyscale, RGB and HSV histograms for all the videos in the directory-based database.
    :return: None
    """
    directory = "/content/Content-Based-Video-Retrieval-Code/footage/"
    files = get_video_filenames(directory)

    # start measuring runtime
    start_time = time.time()

    for file in files:
        if config.model == "gray":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_greyscale_histogram()
        elif config.model == "rgb":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_rgb_histogram()
        elif config.model == "hsv":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_hsv_histogram()
        else:
            histogram_generator_gray = HistogramGenerator(directory, file)
            histogram_generator_gray.generate_video_greyscale_histogram()
            histogram_generator_rgb = HistogramGenerator(directory, file)
            histogram_generator_rgb.generate_video_rgb_histogram()
            histogram_generator_hsv = HistogramGenerator(directory, file)
            histogram_generator_hsv.generate_video_hsv_histogram()
    runtime = round(time.time() - start_time, 2)
    print_finished_training_message(config.model, directory, runtime)


def on_line_retrieval_phase():
    """
    Prompts the user to stabilise and crop the query video before generating the same averaged greyscale, RGB and HSV
    histograms to compare with the database videos' previously stored histograms using distance metrics.
    :return: None
    """
    directory = "/content/Content-Based-Video-Retrieval-Code/recordings/"
    recordings = ["recording1.mp4", "recording2.mp4", "recording3.mp4", "recording4.mp4", "recording5.mp4",
                  "recording6.mp4", "recording7.mp4", "recording8.mp4"]
    mismatches_directory = "/content/Content-Based-Video-Retrieval-Code/recordings/mismatches/"
    mismatches = ["mismatch1.mp4", "mismatch2.mp4"]
    # 0: cloudy-sky, 1: seal, 2: butterfly (skewed), 3: wind-turbine, 4: ice-hockey, 5: jellyfish, 6: people-dancing,
    # 7: jellyfish (skewed)
    file = recordings[7]

    # ask user to stabilise the input query video or not
    is_stabilise_video = terminal_yes_no_question("Do you wish to stabilise the recorded query video?")
    stable_filename = "stable-" + file[:-4] + ".avi"  # the stable version of the video
    # yes: stabilise the video and use the stable .avi version
    if is_stabilise_video:
        if not video_file_already_stabilised(directory + stable_filename):
            VideoStabiliser(directory, "{}".format(file))
        print("\nStabilised version of query already found: '{}'".format(stable_filename))
        file = stable_filename
    # no: check if a version of the stabilised video doesn't already exist - use it if it does
    else:
        if video_file_already_stabilised(directory + stable_filename):
            file = stable_filename

    print("\nUsing query: '{}'".format(file))
    print("\nPlease crop the recorded query video for the signature to be generated.")

    if config.model == "gray":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_greyscale_histogram(is_query=True)
        histogram_generator.match_histograms()
    elif config.model == "rgb":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_rgb_histogram(is_query=True)
        histogram_generator.match_histograms()
    elif config.model == "hsv":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_hsv_histogram(is_query=True)
        histogram_generator.match_histograms()
    else:
        # calculate query histogram
        # greyscale
        histogram_generator_gray = HistogramGenerator(directory, file)
        histogram_generator_gray.generate_video_greyscale_histogram(is_query=True)
        cur_reference_points = histogram_generator_gray.get_current_reference_points()
        # start measuring runtime (after manual cropping)
        start_time = time.time()
        # RGB
        histogram_generator_rgb = HistogramGenerator(directory, file)
        histogram_generator_rgb.generate_video_rgb_histogram(is_query=True, cur_ref_points=cur_reference_points)
        # HSV
        histogram_generator_hsv = HistogramGenerator(directory, file)
        histogram_generator_hsv.generate_video_hsv_histogram(is_query=True, cur_ref_points=cur_reference_points)

        # calculate distances between query and DB histograms
        histogram_generator_gray.match_histograms(cur_all_model='gray')
        histogram_generator_rgb.match_histograms(cur_all_model='rgb')
        histogram_generator_hsv.match_histograms(cur_all_model='hsv')

        # Combine matches from all 3 histogram models to output one final result
        all_results = histogram_generator_hsv.get_results_array()  # array of all matches made (using weights)
        results_count = Counter(all_results)  # count the number of matches made for each video in all_results array
        # transform from count to percentage of matches made
        results_percentage = dict()
        for match in results_count:
            percentage = round((results_count[match] / len(all_results)) * 100.0, 2)
            results_percentage[match] = percentage
        display_results_histogram(results_percentage)
        print("Matches made: {}".format(results_count))
        print("% of matches made: {}".format(results_percentage))

        # find best result
        final_result_name = ""
        final_result_count = 0
        for i, r in enumerate(results_count):
            if i == 0:
                final_result_name = r
                final_result_count = results_count[r]
            else:
                if results_count[r] > final_result_count:
                    final_result_name = r
                    final_result_count = results_count[r]

        # print results
        runtime = round(time.time() - start_time, 2)
        accuracy = final_result_count / len(all_results)
        get_video_first_frame(directory + file, "/content/Content-Based-Video-Retrieval-Code/results", is_query=True)
        get_video_first_frame("/content/Content-Based-Video-Retrieval-Code/footage/{}".format(final_result_name), "../results", is_result=True)
        show_final_match(final_result_name, "/content/Content-Based-Video-Retrieval-Code/results/query.png", "/content/Content-Based-Video-Retrieval-Code/results/result.png", runtime, accuracy)
        print_finished_training_message(final_result_name, config.model, runtime, accuracy)


def database_preprocessing_phase():
    """
    Applies a shot boundary detection algorithm to a video for segmentation.
    :return: None
    """
    directory = "/content/Content-Based-Video-Retrieval-Code/recordings/"
    video = "scene-segmentation.mp4"
    # directory = "/Volumes/ADAM2/"
    # movies = ["Inception (2010).mp4"]
    # video = movies[0]

    shot_boundary_detector = HistogramGenerator(directory, video)
    video_capture = shot_boundary_detector.get_video_capture()
    frame_count = get_number_of_frames(vc=video_capture)
    fps = get_video_fps(vc=video_capture)
    print("Total Frames: {}".format(frame_count))
    print("FPS: {}\n".format(fps))

    # start measuring runtime
    start_time = time.time()

    # start processing video for shout boundary detection
    print("Starting to process video for shot boundary detection...")
    shot_boundary_detector.rgb_histogram_shot_boundary_detection(threshold=7)

    # print final results
    runtime = round(time.time() - start_time, 2)
    print("--- Number of frames in video: {} ---".format(frame_count))
    print("--- Runtime: {} seconds ---".format(runtime))


if __name__ == "__main__":
    main()


In [ ]:
import argparse
from collections import Counter
import time

from pyspin.spin import make_spin, Spin2
import os
import sys

import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from terminaltables import DoubleTable

import numpy as np
from scipy.stats import energy_distance, wasserstein_distance


#import config as config
"""
from helpers import display_results_histogram, get_number_of_frames, get_video_filenames, get_video_fps, \
    get_video_first_frame, print_finished_training_message, terminal_yes_no_question, show_final_match, \
    video_file_already_stabilised
"""
from histogram import HistogramGenerator
from video_operations import VideoStabiliser
#CONFIG
"""
Contains different global variables controlling the system. They set via command line arguments.
"""
model = ""  # variable used to specify the histogram model to use
mode = ""  # variable used to specify the mode to run the program in
show_histograms = False  # Boolean used to decide whether each generated histogram should be displayed or not
debug = False  # Boolean used to print additional logs for debugging purposes
#HELPERS
def get_video_filenames(directory):
    """
    Returns a list containing all the mp4 files in a directory
    :param directory: the directory containing mp4 files
    :return: list of strings
    """
    list_of_videos = list()
    for filename in os.listdir(directory):
        if filename == ".DS_Store":
            pass  # ignoring .DS_Store file (for macOS)
        elif filename.endswith(".mp4"):
            list_of_videos.append(filename)
        else:
            print("no mp4 files found in directory '{}'".format(directory))
    return list_of_videos


def print_terminal_table(table_data, method_used):
    """
    Prints a table with the results in the terminal.
    :param table_data: the data of the table
    :param method_used: the method used, to print as the table title
    :return: None
    """
    table = DoubleTable(table_data)
    table.title = method_used
    table.inner_heading_row_border = False
    table.inner_row_border = True
    print(table.table)


def print_finished_training_message(answer, model, runtime, accuracy=None):
    """
    Prints a message at the end of the training function.
    :param answer: the matched video name
    :param model: the histogram model used for training
    :param runtime: the time elapsed in seconds
    :param accuracy: the accuracy of the classifier in % (True Positives / Number of Matches)
    :return: None
    """
    print("\n\nGenerated " + "\x1b[1;31m" + "{}".format(model) + "\x1b[0m" + " histograms for all videos")
    if accuracy is not None:
        print("\n\n" + "\x1b[1;31m" + "MATCH FOUND: {}".format(answer) + "\x1b[0m")
    print("\n--- Runtime: {} seconds ---".format(runtime))
    if accuracy is not None:
        print("--- Accuracy: {} % ---".format(round(accuracy * 100, 2)))


def get_video_first_frame(video, path_output_dir, is_query=False, is_result=False):
    """
    Retrieves the first frame from a video and saves it as a PNG.
    :param video: the path to the video
    :param path_output_dir: the directory to save the frame in
    :param is_query: write first frame for query
    :param is_result: write first frame for matched video
    :return: None
    """
    vc = cv2.VideoCapture(video)
    frame_counter = 0
    while vc.isOpened():
        ret, image = vc.read()
        if ret and frame_counter == 0:
            if is_query:
                cv2.imwrite(os.path.join(path_output_dir, "query.png"), image)
            elif is_result:
                cv2.imwrite(os.path.join(path_output_dir, "result.png"), image)
            frame_counter += 1
        else:
            break
    cv2.destroyAllWindows()
    vc.release()


def show_final_match(result_name, query_frame, result_frame, runtime, accuracy):
    """
    Plots the query image and the matched video.
    :param result_name: the name of the matched video
    :param query_frame: the query image
    :param result_frame: the matched video's image
    :param runtime: the time elapsed in seconds
    :param accuracy: the accuracy of the classifier in % (True Positives / Number of Matches)
    :return: None
    """
    query_img = mpimg.imread(query_frame)
    result_img = mpimg.imread(result_frame)
    plt.subplot(2, 1, 1)
    plt.imshow(query_img)
    plt.title("Original Query Video", fontSize=16), plt.xticks([]), plt.yticks([])
    plt.subplot(2, 1, 2)
    plt.imshow(result_img)
    plt.title(
        "Match '{}' found in {}s with {}% accuracy".format(result_name, runtime, round(accuracy * 100, 2)),
        fontSize=13)
    plt.xticks([])
    plt.yticks([])
    plt.show()


def display_results_histogram(results_dict):
    """
    Displays the results in the form of a histogram.
    :param results_dict: the histogram with results and the number of matches per video
    :return: None
    """
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.bar(list(results_dict.keys()), results_dict.values())
    plt.title("Probability of a match for most likely videos")
    plt.ylabel("%")
    plt.tight_layout()
    plt.setp(ax.get_xticklabels(), fontsize=10, rotation='vertical')
    plt.show()


def get_number_of_frames(vc):
    """
    Retrieves the total number of frames in a video using OpenCV's VideoCapture object cv2.CAP_PROP_FRAME_COUNT
    attribute.
    :param vc: the video capture
    :return: the number of frames in the video capture
    """
    return int(vc.get(cv2.CAP_PROP_FRAME_COUNT))


def get_video_fps(vc):
    """
    Retrieves the frame rate (Frames Per Second) of a video using OpenCV's VideoCapture object cv2.CAP_PROP_FPS
    attribute.
    :param vc: the video capture
    :return: the video capture's FPS
    """
    return round(vc.get(cv2.CAP_PROP_FPS), 2)


def terminal_yes_no_question(question, default="no"):
    """
    Ask a yes/no question via input() and return the answer as a boolean.
    :param question: string that is presented in the terminal
    :param default: presumed answer if <Enter> directly hit with no answer
    :return: True for "yes" or False for "no"
    """
    valid = {"yes": True, "y": True, "no": False, "n": False}

    if default is None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' (or 'y' or 'n').\n")

#VIDEO_OPERATIONS
from pyspin.spin import make_spin, Box1
from vidstab import VidStab

class ClickAndDrop:
    """
    Class for selecting a region of interest on a frame by click and dropping the mouse over the desired area, and
    cropping that frame to include the pixels inside the ROI only.
    """
    frame_size = (1280, 720)
    window_name = "Crop the recording (top-left click -> bottom-right drop) - 'C' to crop - 'R' to restart"

    def __init__(self, thumbnail):
        """
        Loads the image to crop and controls the callback loop. Calculates the reference points once a valid cropped
        area has been chosen and the user has clicked "c".
        :param thumbnail: the first frame of the video to crop
        """
        self.thumbnail = thumbnail

        self.reference_points = list()
        self.cropping = False

        # load the image, clone it, and setup the mouse callback function
        self.image = cv2.resize(self.thumbnail, self.frame_size, interpolation=cv2.INTER_AREA)
        clone = self.image.copy()
        cv2.namedWindow(self.window_name)
        cv2.setMouseCallback(self.window_name, self.click_and_crop)

        # keep looping until the 'q' key is pressed
        while True:
            # display the image and wait for a keypress
            cv2.imshow(self.window_name, self.image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord("r"):  # reset the cropping region
                self.image = clone.copy()
            elif key == ord("c"):  # break from the loop
                break

        # if there are 2 reference points, then crop the region of interest from the image
        if len(self.reference_points) == 2:
            self.roi = clone[self.reference_points[0][1]:self.reference_points[1][1],
                             self.reference_points[0][0]:self.reference_points[1][0]]

        # close all open windows
        cv2.destroyAllWindows()

    def click_and_crop(self, event, x, y, flags, param):
        """
        Callback function allowing a user to manually crop an image.

        NOTE: must crop from top-left corner to bottom-right corner

        :param event: one of the MouseEventTypes constants
        :param x: the x-coordinate of the mouse event
        :param y: the y-coordinate of the mouse event
        :param flags: one of the MouseEventFlags constants
        :param param: optional parameters
        :return: None
        """
        # if the left mouse button was clicked, record the starting (x, y) coordinates and indicate that cropping is
        # being performed
        if event == cv2.EVENT_LBUTTONDOWN:
            self.reference_points = [(x, y)]
            self.cropping = True

        # check to see if the left mouse button was released
        elif event == cv2.EVENT_LBUTTONUP:
            # record the ending (x, y) coordinates and indicate that the cropping operation is finished
            self.reference_points.append((x, y))
            self.cropping = False

            # draw a rectangle around the region of interest
            cv2.rectangle(self.image, self.reference_points[0], self.reference_points[1], (0, 255, 0), 2)
            cv2.imshow(self.window_name, self.image)

    def get_roi(self):
        """
        Return the selected region of interest of the frame.
        :return: roi: the user-cropped region of the image
        """
        return self.roi

    def get_reference_points(self):
        """
        Returns the ROI's reference points coordinates
        :return: reference_points: a list containing the two x and y coordinates used to crop the image.
        """
        return self.reference_points


class VideoStabiliser:
    """
    Class used to stabilise the recorded video for more optimal matching.
    """
    def __init__(self, directory, file_name):
        """
        Initialise variables and call the function to stabilise the specified video.
        :param directory: the directory where the video file to stabilise is located
        :param file_name: the mp4 video file's name
        """
        self.directory = directory
        self.file = file_name
        self.new_file = file_name[:-4]

        self.stabiliser = VidStab()
        self.stabilise_video()

    @make_spin(Box1, "Stabilising video...")
    def stabilise_video(self):
        """
        Stabilises a mp4 video and outputs the result as an avi file in the same directory.
        :return:
        """
        self.stabiliser.stabilize(input_path="{}{}".format(self.directory, self.file),
                                  output_path="{}/stable-{}.avi".format(self.directory, self.new_file),
                                  border_type="reflect")
        print("\nVideo stabilised!")


#MAIN

def main():
    """
    Program entry point. Parses command line input to decide which phase of the system to run.
    :return: None
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("-m", "--model",
                        help="The histogram model to use. Choose from the following options: 'rgb', 'hsv' or 'gray'. "
                             "Leave empty to train using all 3 histogram models.")
    parser.add_argument("--mode",
                        required=True,
                        help="The mode to run the code in. Choose from the following options: 'train', 'test' or "
                             "'segment'.")
    parser.add_argument("--showhists",
                        action="store_true",
                        help="Specify whether you want to display each generated histogram.")
    parser.add_argument("-d", "--debug",
                        action="store_true",
                        help="Specify whether you want to print additional logs for debugging purposes.")
    args = parser.parse_args()
    config.debug = args.debug
    config.mode = args.mode
    config.show_histograms = args.showhists
    config.model = args.model

    if config.mode == "train":
        off_line_colour_based_feature_extraction_phase()
    elif config.mode == "test":
        on_line_retrieval_phase()
    elif config.mode == "segment":
        database_preprocessing_phase()
    else:
        print("Wrong mode chosen. Choose from the following options: 'train', 'test' or 'segment'.")
        exit(0)


@make_spin(Spin2, "Generating histograms for database videos...".format(config.model))
def off_line_colour_based_feature_extraction_phase():
    """
    Generates and stores averaged greyscale, RGB and HSV histograms for all the videos in the directory-based database.
    :return: None
    """
    directory = "/content/Content-Based-Video-Retrieval-Code/footage/"
    files = get_video_filenames(directory)

    # start measuring runtime
    start_time = time.time()

    for file in files:
        if config.model == "gray":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_greyscale_histogram()
        elif config.model == "rgb":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_rgb_histogram()
        elif config.model == "hsv":
            histogram_generator = HistogramGenerator(directory, file)
            histogram_generator.generate_video_hsv_histogram()
        else:
            histogram_generator_gray = HistogramGenerator(directory, file)
            histogram_generator_gray.generate_video_greyscale_histogram()
            histogram_generator_rgb = HistogramGenerator(directory, file)
            histogram_generator_rgb.generate_video_rgb_histogram()
            histogram_generator_hsv = HistogramGenerator(directory, file)
            histogram_generator_hsv.generate_video_hsv_histogram()
    runtime = round(time.time() - start_time, 2)
    print_finished_training_message(config.model, directory, runtime)


def on_line_retrieval_phase():
    """
    Prompts the user to stabilise and crop the query video before generating the same averaged greyscale, RGB and HSV
    histograms to compare with the database videos' previously stored histograms using distance metrics.
    :return: None
    """
    directory = "/content/Content-Based-Video-Retrieval-Code/recordings/"
    recordings = ["recording1.mp4", "recording2.mp4", "recording3.mp4", "recording4.mp4", "recording5.mp4",
                  "recording6.mp4", "recording7.mp4", "recording8.mp4"]
    mismatches_directory = "/content/Content-Based-Video-Retrieval-Code/recordings/mismatches/"
    mismatches = ["mismatch1.mp4", "mismatch2.mp4"]
    # 0: cloudy-sky, 1: seal, 2: butterfly (skewed), 3: wind-turbine, 4: ice-hockey, 5: jellyfish, 6: people-dancing,
    # 7: jellyfish (skewed)
    file = recordings[7]

    # ask user to stabilise the input query video or not
    is_stabilise_video = terminal_yes_no_question("Do you wish to stabilise the recorded query video?")
    stable_filename = "stable-" + file[:-4] + ".avi"  # the stable version of the video
    # yes: stabilise the video and use the stable .avi version
    if is_stabilise_video:
        if not video_file_already_stabilised(directory + stable_filename):
            VideoStabiliser(directory, "{}".format(file))
        print("\nStabilised version of query already found: '{}'".format(stable_filename))
        file = stable_filename
    # no: check if a version of the stabilised video doesn't already exist - use it if it does
    else:
        if video_file_already_stabilised(directory + stable_filename):
            file = stable_filename

    print("\nUsing query: '{}'".format(file))
    print("\nPlease crop the recorded query video for the signature to be generated.")

    if config.model == "gray":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_greyscale_histogram(is_query=True)
        histogram_generator.match_histograms()
    elif config.model == "rgb":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_rgb_histogram(is_query=True)
        histogram_generator.match_histograms()
    elif config.model == "hsv":
        histogram_generator = HistogramGenerator(directory, file)
        histogram_generator.generate_video_hsv_histogram(is_query=True)
        histogram_generator.match_histograms()
    else:
        # calculate query histogram
        # greyscale
        histogram_generator_gray = HistogramGenerator(directory, file)
        histogram_generator_gray.generate_video_greyscale_histogram(is_query=True)
        cur_reference_points = histogram_generator_gray.get_current_reference_points()
        # start measuring runtime (after manual cropping)
        start_time = time.time()
        # RGB
        histogram_generator_rgb = HistogramGenerator(directory, file)
        histogram_generator_rgb.generate_video_rgb_histogram(is_query=True, cur_ref_points=cur_reference_points)
        # HSV
        histogram_generator_hsv = HistogramGenerator(directory, file)
        histogram_generator_hsv.generate_video_hsv_histogram(is_query=True, cur_ref_points=cur_reference_points)

        # calculate distances between query and DB histograms
        histogram_generator_gray.match_histograms(cur_all_model='gray')
        histogram_generator_rgb.match_histograms(cur_all_model='rgb')
        histogram_generator_hsv.match_histograms(cur_all_model='hsv')

        # Combine matches from all 3 histogram models to output one final result
        all_results = histogram_generator_hsv.get_results_array()  # array of all matches made (using weights)
        results_count = Counter(all_results)  # count the number of matches made for each video in all_results array
        # transform from count to percentage of matches made
        results_percentage = dict()
        for match in results_count:
            percentage = round((results_count[match] / len(all_results)) * 100.0, 2)
            results_percentage[match] = percentage
        display_results_histogram(results_percentage)
        print("Matches made: {}".format(results_count))
        print("% of matches made: {}".format(results_percentage))

        # find best result
        final_result_name = ""
        final_result_count = 0
        for i, r in enumerate(results_count):
            if i == 0:
                final_result_name = r
                final_result_count = results_count[r]
            else:
                if results_count[r] > final_result_count:
                    final_result_name = r
                    final_result_count = results_count[r]

        # print results
        runtime = round(time.time() - start_time, 2)
        accuracy = final_result_count / len(all_results)
        get_video_first_frame(directory + file, "/content/Content-Based-Video-Retrieval-Code/results", is_query=True)
        get_video_first_frame("/content/Content-Based-Video-Retrieval-Code/footage/{}".format(final_result_name), "../results", is_result=True)
        show_final_match(final_result_name, "/content/Content-Based-Video-Retrieval-Code/results/query.png", "/content/Content-Based-Video-Retrieval-Code/results/result.png", runtime, accuracy)
        print_finished_training_message(final_result_name, config.model, runtime, accuracy)


def database_preprocessing_phase():
    """
    Applies a shot boundary detection algorithm to a video for segmentation.
    :return: None
    """
    directory = "/content/Content-Based-Video-Retrieval-Code/recordings/"
    video = "scene-segmentation.mp4"
    # directory = "/Volumes/ADAM2/"
    # movies = ["Inception (2010).mp4"]
    # video = movies[0]

    shot_boundary_detector = HistogramGenerator(directory, video)
    video_capture = shot_boundary_detector.get_video_capture()
    frame_count = get_number_of_frames(vc=video_capture)
    fps = get_video_fps(vc=video_capture)
    print("Total Frames: {}".format(frame_count))
    print("FPS: {}\n".format(fps))

    # start measuring runtime
    start_time = time.time()

    # start processing video for shout boundary detection
    print("Starting to process video for shot boundary detection...")
    shot_boundary_detector.rgb_histogram_shot_boundary_detection(threshold=7)

    # print final results
    runtime = round(time.time() - start_time, 2)
    print("--- Number of frames in video: {} ---".format(frame_count))
    print("--- Runtime: {} seconds ---".format(runtime))


if __name__ == "__main__":
    main()


In [ ]:
import csv
import math
import os

import cv2
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import energy_distance, wasserstein_distance

import app.config as config
from app.helpers import get_video_filenames, print_terminal_table
from app.video_operations import ClickAndDrop


class HistogramGenerator:
    colours = ('b', 'g', 'r')  # RGB channels
    bins = (8, 12, 3)  # 8 hue bins, 12 saturation bins, 3 value bins
    histcmp_methods = [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.HISTCMP_INTERSECT, cv2.HISTCMP_HELLINGER]
    histcmp_3d_methods = ["earths_mover_distance", "energy_distance"]
    histogram_comparison_weigths = {  # weights per comparison methods
        'gray': 1,
        'rgb': 5,
        'hsv': 10
    }
    results_array = list()

    def __init__(self, directory, file_name):
        """
        Initialise variables and create a VideoCapture object for a mp4 file.
        :param directory: the directory where the video file is located
        :param file_name: the mp4 video file's name
        """
        self.directory = directory
        self.file_name = file_name

        # start capturing video
        self.video_capture = cv2.VideoCapture("{}{}".format(self.directory, self.file_name))
        self.check_video_capture()

        # dicts of lists to store histograms for each frame
        self.histograms_grey_dict = list()
        self.histograms_rgb_dict = {
            'b': list(),
            'g': list(),
            'r': list()
        }
        self.histograms_hsv_dict = list()

        # keep current ROI for re-use
        self.reference_points = list()

    def generate_video_rgb_histogram(self, is_query=False, cur_ref_points=None):
        """
        Generates multiple RGB histograms (one every second) for a video.
        :param is_query: boolean specifying if the input video is the query video (to select ROI)
        :param cur_ref_points: list of previously-used ROI point locations
        :return: None
        """
        # determine which frames to process for histograms
        frames_to_process = _get_frames_to_process(self.video_capture)

        frame_counter = 0  # keep track of current frame ID to know to process it or not
        while self.video_capture.isOpened():
            ret, frame = self.video_capture.read()  # read capture frame by frame
            if ret:
                if is_query and frame_counter == 0:
                    if cur_ref_points is None:
                        cad = ClickAndDrop(frame)
                        if config.debug:  # show the cropped region of interest
                            roi_frame = cad.get_roi()
                            cv2.imshow("Selected ROI", roi_frame)
                            cv2.waitKey(0)
                        self.reference_points = cad.get_reference_points()
                    else:
                        self.reference_points = cur_ref_points
                frame_counter += 1
                if frame_counter in frames_to_process:
                    for i, col in enumerate(self.colours):
                        if is_query and len(self.reference_points) == 2:
                            roi = frame[self.reference_points[0][1]:self.reference_points[1][1],
                                        self.reference_points[0][0]:self.reference_points[1][0]]
                            histogram = cv2.calcHist([roi], [i], None, [256], [0, 256])
                        else:
                            histogram = cv2.calcHist([frame], [i], None, [256], [0, 256])
                        self.histograms_rgb_dict[col].append(histogram)
                        if config.debug:  # show individual BGR histogram plots
                            print("i: {}, col: {}".format(i, col))
                            plt.plot(histogram, color=col)
                            plt.xlim([0, 256])
                    if config.debug:
                        plt.show()

                    # user exit on "q" or "Esc" key press
                    k = cv2.waitKey(30) & 0xFF
                    if k == 25 or k == 27:
                        break
            else:
                break
        self.generate_and_store_average_rgb_histogram()
        self.destroy_video_capture()

    def generate_video_greyscale_histogram(self, is_query=False):
        """
        Generates multiple greyscale histograms (one every second) for a video.
        :param is_query: boolean specifying if the input video is the query video (to select ROI)
        :return: None
        """
        # determine which frames to process for histograms
        frames_to_process = _get_frames_to_process(self.video_capture)

        frame_counter = 0  # keep track of current frame ID to know to process it or not
        while self.video_capture.isOpened():
            ret, frame = self.video_capture.read()  # read capture frame by frame
            if ret:
                if is_query and frame_counter == 0:
                    cad = ClickAndDrop(frame)
                    if config.debug:  # show the cropped region of interest
                        roi_frame = cad.get_roi()
                        cv2.imshow("Selected ROI", roi_frame)
                        cv2.waitKey(0)
                    self.reference_points = cad.get_reference_points()
                frame_counter += 1
                if frame_counter in frames_to_process:
                    if is_query and len(self.reference_points) == 2:
                        roi = frame[self.reference_points[0][1]:self.reference_points[1][1],
                                    self.reference_points[0][0]:self.reference_points[1][0]]
                        roi_grey = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                        histogram = cv2.calcHist([roi_grey], [0], None, [256], [0, 256])
                    else:
                        grey_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                        histogram = cv2.calcHist([grey_frame], [0], None, [256], [0, 256])
                    self.histograms_grey_dict.append(histogram)
                    if config.debug:  # show individual greyscale histogram plots
                        plt.figure()
                        plt.title("{} frame {}".format(self.file_name, frame_counter))
                        plt.xlabel("Bins")
                        plt.plot(histogram)
                        plt.xlim([0, 256])
                        plt.show()

                    # user exit on "q" or "Esc" key press
                    k = cv2.waitKey(30) & 0xFF
                    if k == 25 or k == 27:
                        break
            else:
                break
        self.generate_and_store_average_greyscale_histogram()
        self.destroy_video_capture()

    def generate_video_hsv_histogram(self, is_query=False, cur_ref_points=None):
        """
        Generates multiple HSV histograms (one every second) for a video.
        :param is_query: boolean specifying if the input video is the query video (to select ROI)
        :param cur_ref_points: list of previously-used ROI point locations
        :return: None
        """
        # determine which frames to process for histograms
        frames_to_process = _get_frames_to_process(self.video_capture)

        frame_counter = 0  # keep track of current frame ID to know to process it or not
        while self.video_capture.isOpened():
            ret, frame = self.video_capture.read()  # read capture frame by frame
            if ret:
                if is_query and frame_counter == 0:
                    if cur_ref_points is None:
                        cad = ClickAndDrop(frame)
                        if config.debug:  # show the cropped region of interest
                            roi_frame = cad.get_roi()
                            cv2.imshow("Selected ROI", roi_frame)
                            cv2.waitKey(0)
                        self.reference_points = cad.get_reference_points()
                    else:
                        self.reference_points = cur_ref_points
                frame_counter += 1
                if frame_counter in frames_to_process:
                    if is_query and len(self.reference_points) == 2:
                        roi = frame[self.reference_points[0][1]:self.reference_points[1][1],
                                    self.reference_points[0][0]:self.reference_points[1][0]]
                        roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                        histogram = cv2.calcHist([roi_hsv], [0, 1, 2], None, self.bins, [0, 180, 0, 256, 0, 256])
                    else:
                        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                        histogram = cv2.calcHist([hsv_frame], [0, 1, 2], None, self.bins, [0, 180, 0, 256, 0, 256])
                    self.histograms_hsv_dict.append(histogram)
                    if config.debug:  # show individual HSV histogram plots
                        plt.imshow(histogram)
                        plt.title("{} frame {}".format(self.file_name, frame_counter))
                        plt.show()

                    # user exit on "q" or "Esc" key press
                    k = cv2.waitKey(30) & 0xFF
                    if k == 25 or k == 27:
                        break
            else:
                break
        self.generate_and_store_average_hsv_histogram()
        self.destroy_video_capture()

    def generate_and_store_average_rgb_histogram(self):
        """
        Generates a single RGB histogram by averaging all histograms of a video before normalising it and saving the
        results to a plain text file.
        :return: None
        """
        avg_histogram = np.zeros(shape=(255, 1))  # array to store average histogram values
        for col, hists in self.histograms_rgb_dict.items():
            for i in range(0, 255):  # loop through all bins
                bin_sum = 0

                # get value for each colour histogram in bin i
                for arr_index in range(0, len(hists)):
                    bin_value = hists[arr_index].item(i)
                    bin_sum += bin_value

                # average all bins values to store in new histogram
                new_bin_value = bin_sum / len(hists)
                avg_histogram[i] = new_bin_value

            # normalise averaged histogram
            avg_histogram = _normalise_histogram(avg_histogram)

            # write to file
            if not os.path.exists("../histogram_data/{}/".format(self.file_name)):
                os.makedirs("../histogram_data/{}/".format(self.file_name))
            with open("../histogram_data/{}/hist-{}.txt".format(self.file_name, col), 'w') as file:
                file.write("# '{}' channel of RGB histogram ({} bins) [normalised]\n".format(
                    col.upper(),
                    avg_histogram.shape[0]
                ))
                np.savetxt(file, avg_histogram, fmt='%f')
            if config.show_histograms:
                plt.plot(avg_histogram, color=col)
                plt.xlim([0, 256])
        if config.show_histograms:
            plt.title("RGB histogram for '{}'".format(self.file_name))
            plt.xlabel("Bins")
            plt.show()

    def generate_and_store_average_greyscale_histogram(self):
        """
        Generates a single greyscale histogram by averaging all histograms of a video before normalising it and saving
        the results to a plain text file.
        :return: None
        """
        avg_histogram = np.zeros(shape=(255, 1))  # array to store average histogram values

        col = "gray"
        hist = self.histograms_grey_dict

        for i in range(0, 255):  # loop through all bins
            bin_sum = 0

            # get value for each colour histogram in bin i
            for arr_index in range(0, len(hist)):
                bin_value = hist[arr_index].item(i)
                bin_sum += bin_value

            # average all bins values to store in new histogram
            new_bin_value = bin_sum / len(hist)
            avg_histogram[i] = new_bin_value

        # normalise averaged histogram
        avg_histogram = _normalise_histogram(avg_histogram)

        # write to file
        if not os.path.exists("../histogram_data/{}/".format(self.file_name)):
            os.makedirs("../histogram_data/{}/".format(self.file_name))
        with open("../histogram_data/{}/hist-{}.txt".format(self.file_name, col), 'w') as file:
            file.write("# Greyscale Histogram ({} bins) [normalised]\n".format(avg_histogram.shape[0]))
            np.savetxt(file, avg_histogram, fmt='%f')
        if config.show_histograms:
            plt.plot(avg_histogram, color=col)
            plt.xlim([0, 256])
            plt.title("Greyscale histogram for '{}'".format(self.file_name))
            plt.xlabel("Bins")
            plt.show()

    def generate_and_store_average_hsv_histogram(self):
        """
        Generates a single HSV histogram by averaging all histograms of a video before normalising it and saving the
        results to a plain text file.
        :return: None
        """
        avg_histogram = np.zeros(shape=(8, 12, 3))  # array to store average histogram values

        col = "hsv"
        hist = self.histograms_hsv_dict

        for h in range(0, self.bins[0]):  # loop through hue bins
            for s in range(0, self.bins[1]):  # loop through saturation bins
                for v in range(0, self.bins[2]):  # loop through value bins
                    bin_sum = 0

                    # get value for each colour histogram in bin [h][s][v]
                    for arr_index in range(0, len(hist)):
                        bin_value = hist[arr_index][h][s][v]
                        bin_sum += bin_value

                    # average all bins values to store in new histogram
                    new_bin_value = bin_sum / len(hist)
                    avg_histogram[h][s][v] = new_bin_value

        # normalise averaged histogram
        avg_histogram = _normalise_histogram(avg_histogram)

        # write to file
        if not os.path.exists("../histogram_data/{}/".format(self.file_name)):
            os.makedirs("../histogram_data/{}/".format(self.file_name))
        with open("../histogram_data/{}/hist-{}.txt".format(self.file_name, col), 'w') as file:
            file.write("# HSV Histogram shape: {0} [normalised]\n".format(avg_histogram.shape))
            for arr_2d in avg_histogram:
                file.write("# New slice\n")
                np.savetxt(file, arr_2d)

        if config.show_histograms:
            plt.imshow(avg_histogram)
            plt.title("HSV histogram for '{}'".format(self.file_name))
            plt.xlabel("Hue")
            plt.ylabel("Saturation")
            plt.show()

    def match_histograms(self, cur_all_model="all"):
        """
        Compares the greyscale, RGB and HSV histograms of the query video with each of the saved average histograms
        using different distance metrics such as the Correlation, Intersection, Chi-Square Distance, Hellinger Distance,
        Earth's Mover Distance and Energy Distance metrics. Finally, prints the results for each histogram model and
        metric in a console table and writes the data to a CSV file.
        :param cur_all_model: the current histogram model when operating with all 3 models
        :return: None
        """
        # variables used for finding the match to the recorded video
        video_match = ""
        video_match_value = 0

        # get histogram for the recorded video to match - todo: calculate the histogram on the go
        query_histogram = dict()
        if config.model == "gray" or (cur_all_model == "gray" and config.model == "all"):
            query_histogram = {
                'gray': np.loadtxt(
                    "../histogram_data/{}/hist-gray.txt".format(self.file_name),
                    dtype=np.float32,
                    unpack=False
                )
            }
        elif config.model == "rgb" or (cur_all_model == "rgb" and config.model == "all"):
            query_histogram = {
                'b': np.loadtxt("../histogram_data/{}/hist-b.txt".format(self.file_name), dtype=np.float32, unpack=False),
                'g': np.loadtxt("../histogram_data/{}/hist-g.txt".format(self.file_name), dtype=np.float32, unpack=False),
                'r': np.loadtxt("../histogram_data/{}/hist-r.txt".format(self.file_name), dtype=np.float32, unpack=False)
            }
        elif config.model == "hsv" or (cur_all_model == "hsv" and config.model == "all"):
            hsv_data = np.loadtxt("../histogram_data/{}/hist-hsv.txt".format(self.file_name))
            query_histogram = {
                'hsv': hsv_data.reshape((8, 12, 3))
            }

        # compare query histogram with each DB video histogram
        print("\n{} Histogram Comparison Results:\n".format(_get_chosen_model_string(cur_all_model)))

        method = ""
        csv_field_names = ["video", "score"]

        # use OpenCV's compareHist function for RGB and greyscale histograms (works with 2d arrays only)
        if config.model == "rgb" \
                or config.model == "gray" \
                or (cur_all_model == "gray" and config.model == "all") \
                or (cur_all_model == "rgb" and config.model == "all"):
            for m in self.histcmp_methods:
                if m == 0:
                    method = "CORRELATION"
                elif m == 1:
                    method = "CHI-SQUARE"
                elif m == 2:
                    method = "INTERSECTION"
                elif m == 3:
                    method = "HELLINGER"

                # CSV file to write data to for each method
                if config.model == "all":
                    csv_file = open("../results/csv/{}-{}-{}.csv".format(config.model, cur_all_model, method), 'w')
                else:
                    csv_file = open("../results/csv/{}-{}.csv".format(config.model, method), 'w')
                with csv_file:
                    writer = csv.DictWriter(csv_file, fieldnames=csv_field_names)
                    writer.writeheader()

                    table_data = list()
                    for i, file in enumerate(get_video_filenames("../footage/")):
                        comparison = 0
                        if config.model == "gray" or (cur_all_model == "gray" and config.model == "all"):
                            dbvideo_greyscale_histogram = np.loadtxt("../histogram_data/{}/hist-gray.txt".format(file), dtype=np.float32, unpack=False)
                            comparison = cv2.compareHist(query_histogram['gray'], dbvideo_greyscale_histogram, m)
                        elif config.model == "rgb" or (cur_all_model == "rgb" and config.model == "all"):
                            dbvideo_b_histogram = np.loadtxt("../histogram_data/{}/hist-b.txt".format(file), dtype=np.float32, unpack=False)
                            dbvideo_g_histogram = np.loadtxt("../histogram_data/{}/hist-g.txt".format(file), dtype=np.float32, unpack=False)
                            dbvideo_r_histogram = np.loadtxt("../histogram_data/{}/hist-r.txt".format(file), dtype=np.float32, unpack=False)
                            comparison_b = cv2.compareHist(query_histogram['b'], dbvideo_b_histogram, m)
                            comparison_g = cv2.compareHist(query_histogram['g'], dbvideo_g_histogram, m)
                            comparison_r = cv2.compareHist(query_histogram['r'], dbvideo_r_histogram, m)
                            comparison = (comparison_b + comparison_g + comparison_r) / 3

                        # append data to table
                        table_data.append([file, round(comparison, 5)])

                        # write data to CSV file
                        writer.writerow({"video": file, "score": round(comparison, 5)})

                        if i == 0:
                            video_match = file
                            video_match_value = comparison
                        else:
                            # Higher score = better match (Correlation and Intersection)
                            if m in [0, 2] and comparison > video_match_value:
                                video_match = file
                                video_match_value = comparison
                            # Lower score = better match
                            # (Chi-square, Alternative chi-square, Hellinger and Kullback-Leibler Divergence)
                            elif m in [1, 3, 4, 5] and comparison < video_match_value:
                                video_match = file
                                video_match_value = comparison

                # append video match found to results list (using weights)
                if cur_all_model == "gray":
                    for _ in range(0, self.histogram_comparison_weigths['gray'], 1):
                        self.results_array.append(video_match)
                elif cur_all_model == "rgb":
                    for _ in range(0, self.histogram_comparison_weigths['rgb'], 1):
                        self.results_array.append(video_match)

                print_terminal_table(table_data, method)
                print("{} {} match found: ".format(_get_chosen_model_string(cur_all_model), method) +
                      "\x1b[1;31m" + video_match + "\x1b[0m" + "\n\n")

        # use SciPy's statistical distances functions for HSV histograms (compareHist does not work with 3d arrays)
        elif config.model == "hsv" or config.model == "all":
            for m in self.histcmp_3d_methods:
                if m == "earths_mover_distance":
                    method = "EARTH'S MOVER DISTANCE"
                elif m == "energy_distance":
                    method = "ENERGY DISTANCE"

                # CSV file to write data to for each method
                if config.model == "all":
                    csv_file = open("../results/csv/{}-{}-{}.csv".format(config.model, cur_all_model, method), 'w')
                else:
                    csv_file = open("../results/csv/{}-{}.csv".format(config.model, method), 'w')
                with csv_file:

                    writer = csv.DictWriter(csv_file, fieldnames=csv_field_names)
                    writer.writeheader()

                    table_data = list()
                    for i, file in enumerate(get_video_filenames("../footage/")):
                        dbvideo_hsv_histogram_data = np.loadtxt("../histogram_data/{}/hist-hsv.txt".format(file))
                        dbvideo_hsv_histogram = dbvideo_hsv_histogram_data.reshape((8, 12, 3))
                        comparison = 0
                        for h in range(0, self.bins[0]):  # loop through hue bins
                            for s in range(0, self.bins[1]):  # loop through saturation bins
                                query_histogram_slice = query_histogram['hsv'][h][s]
                                dbvideo_histogram_slice = dbvideo_hsv_histogram[h][s]
                                if method == "EARTH'S MOVER DISTANCE":
                                    comparison += wasserstein_distance(query_histogram_slice, dbvideo_histogram_slice)
                                elif method == "ENERGY DISTANCE":
                                    comparison += energy_distance(query_histogram_slice, dbvideo_histogram_slice)

                        # append data to table
                        table_data.append([file, round(comparison, 5)])

                        # write data to CSV file
                        writer.writerow({"video": file, "score": round(comparison, 5)})

                        if i == 0:
                            video_match = file
                            video_match_value = comparison
                        else:
                            if comparison < video_match_value:
                                video_match = file
                                video_match_value = comparison

                # append video match found to results list (using weights)
                for _ in range(0, self.histogram_comparison_weigths['hsv']):
                    self.results_array.append(video_match)

                print_terminal_table(table_data, method)
                print("{} {} Match found: ".format(_get_chosen_model_string(cur_all_model), method) +
                      "\x1b[1;31m" + video_match + "\x1b[0m" + "\n\n")

    def rgb_histogram_shot_boundary_detection(self, threshold):
        """
        Compares consecutive frames' RGB histograms using the Intersection metric with a global threshold approach.
        If the metric is smaller than the specified threshold, then a shot boundary has been detected.
        :param threshold: integer specifying the global threshold for the algorithm
        :return: None
        """
        x_axis = list()
        y_axis = list()
        is_under_threshold = True

        ret, frame = self.video_capture.read()  # get initial frame

        frame_counter = 0  # keep track of current frame ID to locate shot boundaries
        shot_changes_detected = 0  # keep track of the number of shot changes detected
        while self.video_capture.isOpened():
            prev_frame = frame[:]  # previous frame
            ret, frame = self.video_capture.read()  # read capture frame by frame

            if ret:
                frame_counter += 1
                cur_rgb_hist = {
                    'b': list(),
                    'g': list(),
                    'r': list()
                }
                prev_rgb_hist = {
                    'b': list(),
                    'g': list(),
                    'r': list()
                }
                for i, col in enumerate(self.colours):
                    # calculate RGB histograms
                    cur_frame_hist = cv2.calcHist([frame], [i], None, [256], [0, 256])
                    prev_frame_hist = cv2.calcHist([prev_frame], [i], None, [256], [0, 256])

                    # normalise histograms
                    cur_frame_hist = _normalise_histogram(cur_frame_hist)
                    prev_frame_hist = _normalise_histogram(prev_frame_hist)

                    # save histograms in dict
                    cur_rgb_hist[col].append(cur_frame_hist)
                    prev_rgb_hist[col].append(prev_frame_hist)

                # calculate Intersection between consecutive frames
                comparison_r = cv2.compareHist(prev_rgb_hist['r'][0], cur_rgb_hist['r'][0], cv2.HISTCMP_INTERSECT)
                comparison_g = cv2.compareHist(prev_rgb_hist['g'][0], cur_rgb_hist['g'][0], cv2.HISTCMP_INTERSECT)
                comparison_b = cv2.compareHist(prev_rgb_hist['b'][0], cur_rgb_hist['b'][0], cv2.HISTCMP_INTERSECT)
                comparison = (comparison_b + comparison_g + comparison_r) / 3
                # For KL Divergence, use cv2.HISTCMP_KL_DIV

                # append data to lists for plot
                x_axis.append(frame_counter)
                y_axis.append(comparison)

                if comparison < threshold and is_under_threshold:
                    shot_changes_detected += 1
                    is_under_threshold = False
                    print("Scene Change detected at Frame {}".format(frame_counter))
                elif comparison > threshold:
                    is_under_threshold = True

            else:
                break

        # Plot results
        plt.plot(x_axis, y_axis)
        plt.plot(x_axis, np.full(frame_counter, threshold))
        plt.title("Intersection Between Consecutive Frame RGB Histogram")
        plt.xlabel("Frame")
        plt.ylabel("Intersection")
        plt.show()
        print("\n--- Number of shot changes detected: {} ---".format(shot_changes_detected))

        self.destroy_video_capture()

    def check_video_capture(self):
        """
        Checks if the VideoCapture object was correctly created.
        :return: None
        """
        if not self.video_capture.isOpened():
            print("Error opening video file")

    def destroy_video_capture(self):
        """
        Tidying up the OpenCV environment and the video capture.
        :return: None
        """
        self.video_capture.release()
        cv2.destroyAllWindows()

    def get_video_capture(self):
        """
        Returns the full VideoCapture object.
        :return: the VideoCapture object
        """
        return self.video_capture

    def get_current_reference_points(self):
        """
        Returns the current ROI point locations manually selected for the first frame for future re-use.
        :return: the ROI pixel locations retrieved from the first frame of the video
        """
        return self.reference_points

    def get_results_array(self):
        """
        Returns the array with the resulting video results.
        :return: array of strings
        """
        return self.results_array


def _normalise_histogram(hist):
    """
    Normalise a histogram using OpenCV's "normalize" function.
    :param hist: the histogram to normalise
    :return: the normalised histogram
    """
    hist = cv2.normalize(hist, hist)
    return hist


def _get_frames_to_process(vc):
    """
    Returns the IDs of the frames to calculate a histogram for. 1 Frame Per Second.
    :param vc: the VideoCapture object to process
    :return: a list of integers representing the frames to process
    """
    frame_ids = list()
    total_frames = vc.get(cv2.CAP_PROP_FRAME_COUNT)
    fps = vc.get(cv2.CAP_PROP_FPS)
    for i in range(1, int(total_frames) + 1, math.ceil(fps)):
        frame_ids.append(i)
    return frame_ids


def _get_chosen_model_string(model):
    """
    Returns the Histogram Model chosen for the matching process.
    :return: a string representing the chosen histogram model
    """
    if model == "gray":
        return "Greyscale"
    elif model == "rgb":
        return "RGB"
    elif model == "hsv":
        return "HSV"
    else:
        if config.model == "gray":
            return "Greyscale"
        elif config.model == "rgb":
            return "RGB"
        elif config.model == "hsv":
            return "HSV"
